**Import Required Libraries**

In [39]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
import numpy as np
from deepface import DeepFace
import pandas as pd
from sklearn.model_selection import train_test_split
import wandb
import cv2
from wandb.keras import WandbCallback

In [ ]:
wandb.init(project="Actor Detection")

**Data**

In [42]:
emb_list = []
cat_list = []

for i in range(3):
  for j in range(1,21):
    embedding = DeepFace.represent(img_path = f"input\{i}"f" ({j}).jpg", model_name = 'Facenet')
    emb_list.append(embedding)
    cat_list.append(i)

In [43]:
df = pd.DataFrame(emb_list)
df['category'] = cat_list
df.to_csv('feature_list.csv', index=False)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,category
0,0.705229,-0.467777,-0.849564,1.631884,0.781920,-1.074667,0.605650,-0.553223,0.463141,1.357267,...,0.089603,0.087537,1.491058,0.300430,-1.436116,-0.676035,0.206691,-1.134410,0.895477,0
1,-0.332985,-0.330223,-0.866652,1.594688,2.301664,0.116573,-1.522974,1.932201,1.022533,1.514163,...,-0.373162,-0.048413,0.704640,-0.371660,-2.395536,0.188537,-1.178383,0.112215,0.987411,0
2,0.290644,0.950106,-0.213829,1.623187,1.202535,-0.308924,1.210230,-0.343003,0.627491,1.614541,...,0.037902,0.677695,0.871504,-0.598366,-2.574017,-0.332315,-0.447244,-0.514461,-0.148675,0
3,-0.468002,-0.292499,-0.836633,0.629525,1.532991,-0.744017,-0.868493,0.063227,1.062264,1.181333,...,-0.227405,-0.242751,1.348131,0.502554,-2.490005,0.318960,-0.464727,-0.271058,0.054087,0
4,-0.031010,-0.418403,-0.310325,1.643237,1.757546,-0.835343,0.569866,0.098678,0.711295,1.898196,...,0.165184,0.526260,1.350363,-0.508886,-0.633738,0.203807,-0.785905,-0.245729,-0.003731,0


In [ ]:
X = df.iloc[0:,0:128].values
Y = df['category'].values

In [45]:
X_train,X_test,Y_train,Y_test = train_test_split(X ,Y ,shuffle=True)

In [46]:
config = wandb.config
config.learning_rate = 0.1

**Model**

In [47]:
model = tf.keras.models.Sequential([
    Dense(127 ,activation="sigmoid"),
    Dense(128, activation="relu"),
    Dense(256, activation="relu"),
    Dense(3, activation="softmax") 
])

In [48]:
model.compile(optimizer=tf.keras.optimizers.SGD(config.learning_rate),
              loss = tf.keras.losses.sparse_categorical_crossentropy,
              metrics = ['accuracy'])

In [ ]:
model.fit(X_train, Y_train, epochs=150, validation_data=(X_test, Y_test),callbacks=[WandbCallback()])

In [50]:
model.evaluate(X_test, Y_test)

1/1 [==============================] - 0s 24ms/step - loss: 0.0490 - accuracy: 1.0000


[0.04903601109981537, 1.0]

**Test**

In [51]:
test_img= DeepFace.represent(img_path = 'input\0 (55).jpg' , model_name = 'Facenet')

result = model.predict([test_img])
np.argmax(result)

0